# 📊 Bài A2: Dự đoán giá trị liên tục với Boston Housing
---
**Yêu cầu:**
- Phân tích dữ liệu
- Tiền xử lý đặc trưng
- Huấn luyện các mô hình nâng cao
- Đánh giá mô hình chuyên sâu


In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.neural_network import MLPRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
import optuna
import shap
import numpy as np
import warnings
warnings.filterwarnings("ignore")

url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
df = pd.read_csv(url)

print(df.head())
print(df.info())
print(df.describe())


In [ ]:

sns.pairplot(df[['rm', 'lstat', 'ptratio', 'tax', 'price']])
plt.show()


In [ ]:

corr_matrix = df.corr(method='pearson')
high_corr = corr_matrix['price'].abs().sort_values(ascending=False)
print("Các biến tương quan mạnh với price:\n", high_corr[high_corr > 0.5])


In [ ]:

iso = IsolationForest(contamination=0.05, random_state=42)
outliers = iso.fit_predict(df)
df_clean = df[outliers == 1].reset_index(drop=True)
print("Dữ liệu còn lại sau khi loại outlier:", df_clean.shape)


In [ ]:

X = df_clean.drop(columns=['price'])
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)


In [ ]:

df_clean['room_per_crime'] = df_clean['rm'] / df_clean['crim']
df_clean['high_tax'] = (df_clean['tax'] > df_clean['tax'].mean()).astype(int)
df_clean['rm_lstat'] = df_clean['rm'] * df_clean['lstat']
df_clean['ptratio_tax'] = df_clean['ptratio'] * df_clean['tax']


In [ ]:

features_to_poly = ['rm', 'lstat', 'ptratio']
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(df_clean[features_to_poly])
poly_feature_names = poly.get_feature_names_out(features_to_poly)
poly_df = pd.DataFrame(poly_features, columns=poly_feature_names)
df_clean = pd.concat([df_clean.reset_index(drop=True), poly_df], axis=1)


In [ ]:

X = df_clean.drop(columns=['price'])
y = df_clean['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(X_train_scaled, y_train)

mlp = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)


In [ ]:

def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10)
    }
    model = GradientBoostingRegressor(**params, random_state=42)
    score = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
    return -score.mean()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print("Best parameters:", study.best_params)


In [ ]:

estimators = [
    ('lr', lr),
    ('gbr', GradientBoostingRegressor(**study.best_params, random_state=42)),
    ('mlp', mlp)
]

stack = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stack.fit(X_train_scaled, y_train)


In [ ]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_model(model, X, y):
    mse = -cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv).mean()
    rmse = np.sqrt(mse)
    r2 = cross_val_score(model, X, y, scoring='r2', cv=cv).mean()
    mape = -cross_val_score(model, X, y, scoring='neg_mean_absolute_percentage_error', cv=cv).mean()
    return mse, rmse, r2, mape

models = {'Linear Regression': lr, 'Gradient Boosting': gbr, 'MLP Regressor': mlp, 'Stacking': stack}

for name, model in models.items():
    mse, rmse, r2, mape = evaluate_model(model, X_train_scaled, y_train)
    print(f"{name}: MSE={mse:.2f}, RMSE={rmse:.2f}, R²={r2:.3f}, MAPE={mape:.2f}")


In [ ]:

y_pred = stack.predict(X_test_scaled)
plt.scatter(y_test, y_test - y_pred)
plt.axhline(0, color='red')
plt.xlabel('Actual price')
plt.ylabel('Residuals')
plt.title('Residual Plot - Stacking Model')
plt.show()


In [ ]:

explainer = shap.Explainer(gbr, X_train_scaled)
shap_values = explainer(X_test_scaled)
shap.summary_plot(shap_values, X_test, plot_type="bar")
